In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### UCF DATA

In [ ]:
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
import copy
import os
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
import cv2
import sys

In [ ]:
import os

#Label file:
data_path = '/kaggle/input/ucf-crime-dataset/Train'
classes = os.listdir(data_path)

In [ ]:
classes

### Create Directory

In [ ]:
import re
import os

def extract_numbers_from_filename(filename, class_name):
    pattern = f'{class_name}(\d+)_'
    match = re.search(pattern, filename)
    if match:
        first_number = int(match.group(1))
    else:
        first_number = -1


    second_match = re.search(r'_(\d+)\.png', filename)
    if second_match:
        second_number = int(second_match.group(1))
    else:
        second_number = -1

    return first_number, second_number


def get_sorted_filenames(directory_path, class_names):
    sorted_filenames = {}

    for class_name in class_names:
        if class_name == 'NormalVideos':
            file_names = os.listdir(os.path.join(directory_path, class_name))
            sorted_filenames[class_name] = sorted(file_names, key=lambda x: extract_numbers_from_filename(x, 'Normal_Videos'))
        else:
            file_names = os.listdir(os.path.join(directory_path, class_name))
            sorted_filenames[class_name] = sorted(file_names, key=lambda x: extract_numbers_from_filename(x, class_name))

    return sorted_filenames


train_path = "/kaggle/input/ucf-crime-dataset/Train/"
val_path = "/kaggle/input/ucf-crime-dataset/Test/"

train_class = ['Assault', 'NormalVideos', 'Shoplifting', 'Robbery', 'Stealing', 'Burglary', 'Fighting']

train_sorted_filenames = get_sorted_filenames(train_path, train_class)

val_sorted_filenames = get_sorted_filenames(val_path, train_class)

In [ ]:
train_sorted_filenames['NormalVideos'][:10]

In [ ]:
val_sorted_filenames['NormalVideos'][:10]

#### 하위디렉토리 추가

#### 하위 디렉토리별 이미지 생성

In [ ]:

import os

def create_subdirectories(sorted_filenames, output_dir, max_count_per_class):
    for class_name, filenames in sorted_filenames.items():
        max_count = 0

        for filename in filenames:
            # NormalVideos 클래스의 파일명 패턴에 따라 처리
            if class_name == 'NormalVideos':
                subdir_name = filename.split('_x264')[0]
            else:
                subdir_name = filename.split('_')[0]

            subdir_path = os.path.join(output_dir, class_name)
            subdir_path = os.path.join(subdir_path, subdir_name)

            if max_count < max_count_per_class:
                if not os.path.exists(subdir_path):
                    os.makedirs(subdir_path)
                    max_count += 1



train_out = "/kaggle/working/ucf-crime-dataset/Train/"
val_out = "/kaggle/working/ucf-crime-dataset/Test"

max_count_per_class = 50  # 하위 디렉토리 최대 개수
create_subdirectories(train_sorted_filenames, train_out, max_count_per_class)
create_subdirectories(val_sorted_filenames, val_out, max_count_per_class)



In [ ]:
import os
import shutil

def copy_images_to_subdirectories(sorted_filenames, source_dir, output_dir):
    for class_name, filenames in sorted_filenames.items():
        for filename in filenames:
            # NormalVideos 클래스의 파일명 패턴에 따라 처리
            if class_name == 'NormalVideos':
                subdir_name = filename.split('_x264')[0]
            else:
                subdir_name = filename.split('_')[0]

            subdir_path = os.path.join(output_dir, class_name)
            subdir_path = os.path.join(subdir_path, subdir_name)

            if os.path.exists(subdir_path):
                # 이미지 파일 복사
                src_path = os.path.join(source_dir, class_name, filename)
                dest_path = os.path.join(subdir_path, filename)
                shutil.copy(src_path, dest_path)

# 사용 예시
train_source_dir = "/kaggle/input/ucf-crime-dataset/Train/"
train_output_dir = "/kaggle/working/ucf-crime-dataset/Train"

val_source_dir = "/kaggle/input/ucf-crime-dataset/Test/"
val_output_dir = "/kaggle/working/ucf-crime-dataset/Test"

copy_images_to_subdirectories(train_sorted_filenames, train_source_dir, train_output_dir)
copy_images_to_subdirectories(val_sorted_filenames, val_source_dir, val_output_dir)



In [ ]:
import shutil

directory_to_remove = "/kaggle/working/ucf-crime-dataset/Test"  # 삭제할 디렉토리 경로
shutil.rmtree(directory_to_remove)

print(f"디렉토리 {directory_to_remove}를 제거하였습니다.")

### SlowFastDataset

In [ ]:
class SlowFastDataset(Dataset):
    def __init__(self, data_dir, train_classes, transform=None):
        self.data_dir = data_dir
        self.train_classes = train_classes
        self.transform = transform

        self.data = self._load_data()

    def _load_data(self):
        data = []
        for class_name in self.train_classes:
            class_dir = os.path.join(self.data_dir, class_name)
            for video_dir in os.listdir(class_dir):
                video_path = os.path.join(class_dir, video_dir)
                frames = os.listdir(video_path)
                if len(frames) >= 100:
                    data.append((video_path, class_name))
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        video_path, class_name = self.data[idx]
        frames = os.listdir(video_path)

        # Slow 프레임 와 Fast 프레임의 비율 8:2
        slow_frames = random.sample(frames, 8*10)
        fast_frames = random.sample(frames, 2*10)

        slow_paths = [os.path.join(video_path, frame) for frame in slow_frames]
        fast_paths = [os.path.join(video_path, frame) for frame in fast_frames]

        slow_images = [self.transform(Image.open(path)) for path in slow_paths]
        fast_images = [self.transform(Image.open(path)) for path in fast_paths]

        # Convert to tensors and stack for 5D tensor
        slow_images = torch.stack(slow_images, dim=0).permute(1, 0, 2, 3)  # (frames, batch_size, channels, height, width)
        fast_images = torch.stack(fast_images, dim=0).permute(1, 0, 2, 3)  # (frames, batch_size, channels, height, width)

        sample = {
            'slow_images': slow_images,
            'fast_images': fast_images,
            'class_name': class_name
        }
        return sample




#### 데이터셋 및 데이터로더 생성

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
CONFIG = {
    'FPS':30,
    'IMG_SIZE': 224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':4,
    'SEED':42
}

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

set_seed(CONFIG['SEED'])

In [ ]:
transform = transforms.Compose([
    transforms.Resize((CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE'])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# Train Dataset
train_data_dir = '/kaggle/working/ucf-crime-dataset/Train'
train_classes = ['Assault', 'NormalVideos', 'Shoplifting', 'Robbery', 'Stealing', 'Burglary', 'Fighting']

train_dataset = SlowFastDataset(train_data_dir, train_classes, transform)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)


# Validation Dataset
val_data_dir = '/kaggle/working/ucf-crime-dataset/Test'
val_dataset = SlowFastDataset(val_data_dir, train_classes, transform)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=True)



# 데이터 확인
for batch in val_dataloader:
    slow_images = batch['slow_images']
    fast_images = batch['fast_images']
    class_names = batch['class_name']

    print('Slow images shape:', slow_images[0].shape)
    print('Fast images shape:', fast_images[0].shape)
    print('Class names:', class_names)
    break  # 첫 번째 배치만 확인



Slow images shape: torch.Size([3, 80, 224, 224])
Fast images shape: torch.Size([3, 20, 224, 224])
Class names: ['NormalVideos', 'NormalVideos', 'NormalVideos', 'NormalVideos']


In [ ]:
print('Train : ',len(train_dataset))
print('Val : ',len(val_dataset))

Train :  264
Val :  86


### Train

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [ ]:
model = models.video.r3d_18(pretrained=True) # https://pytorch.org/vision/0.8/models.html
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(train_classes))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader

def train_and_validate(model, train_loader, val_loader, num_epochs, device):
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for batch in train_loader:
            slow_images = batch['slow_images']
            fast_images = batch['fast_images']
            labels = batch['class_name']
            print(labels)
            slow_images = slow_images.to(device)
            fast_images = fast_images.to(device)
            labels = labels

            optimizer.zero_grad()

            slow_outputs = model(slow_images)
            fast_outputs = model(fast_images)

            slow_loss = criterion(slow_outputs, labels)
            fast_loss = criterion(fast_outputs, labels)
            loss = slow_loss + fast_loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}] - Train Loss: {epoch_loss:.4f}')

        # Validation
        model.eval()
        val_loss = 0.0
        correct = 0

        with torch.no_grad():
            for batch in val_loader:
                slow_images = batch['slow_images'].to(device)
                fast_images = batch['fast_images'].to(device)
                labels = batch['class_name'].to(device)

                slow_outputs = model(slow_images)
                fast_outputs = model(fast_images)

                slow_loss = criterion(slow_outputs, labels)
                fast_loss = criterion(fast_outputs, labels)
                loss = slow_loss + fast_loss

                val_loss += loss.item()

                _, slow_predicted = slow_outputs.max(1)
                _, fast_predicted = fast_outputs.max(1)
                combined_predicted = (slow_predicted + fast_predicted) // 2
                correct += combined_predicted.eq(labels).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = correct / len(val_loader.dataset)
        print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

    print('Training finished!')


train_and_validate(model, train_dataloader, val_dataloader, 5, device)


['Stealing', 'Stealing', 'NormalVideos', 'Fighting']


OutOfMemoryError: CUDA out of memory. Tried to allocate 246.00 MiB (GPU 0; 14.76 GiB total capacity; 13.74 GiB already allocated; 65.75 MiB free; 13.78 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)

    best_val_score = 0
    best_model = None

    for epoch in range(1, CONFIG['EPOCHS']+1):
        model.train()
        train_loss = []
        for videos, labels in tqdm(iter(train_loader)):
            videos = videos.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            output = model(videos)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')

        if scheduler is not None:
            scheduler.step(_val_score)

        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model

    return best_model



In [ ]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []

    with torch.no_grad():
        for videos, labels in tqdm(iter(val_loader)):
            videos = videos.to(device)
            labels = labels.to(device)

            logit = model(videos)

            loss = criterion(logit, labels)

            val_loss.append(loss.item())

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()

        _val_loss = np.mean(val_loss)

    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

In [ ]:
model = models.video.r3d_18(pretrained=True) # https://pytorch.org/vision/0.8/models.html
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(train_classes))


In [ ]:
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CONFIG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_dataloader, val_dataloader, scheduler, device)


### slowfast_8x8_resnet50_kinetics400

In [ ]:
# 모델 로드
model = models.video.r3d_18(pretrained=True)
num_classes = len(train_classes)  # 클래스 수에 맞게 설정

# 손실 함수 정의 (예: CrossEntropyLoss)
criterion = nn.CrossEntropyLoss()

# 옵티마이저 정의 (예: Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# 학습
num_epochs = 10  # 적절한 학습 에포크 수로 설정
for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 변경
    running_loss = 0.0

    for batch in dataloader:
        inputs = batch['frames']
        labels = batch['class_name']

        # 전처리 및 데이터를 모델에 전달
        inputs = torch.stack([transform(frame) for frame in inputs])
        outputs = model(inputs)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 역전파 및 가중치 갱신
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{num_epochs}] - Loss: {epoch_loss:.4f}')

print('Training finished!')

In [ ]:
import torch

print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

In [ ]:
from gluoncv import model_zoo

# Load the SlowFast model
model = model_zoo.get_model('slowfast_8x8_resnet50_kinetics400', pretrained=True)


In [ ]:
pip install decord --user

In [ ]:
import shutil
import os

# 원본 데이터 디렉토리 경로와 로컬 환경에 저장할 경로 설정
source_dir = "/kaggle/input/ucf-crime-dataset"
local_dir = "/kaggle/working/ucf-crime-dataset"  # 로컬 디렉토리 경로 변경 가능

# 데이터 복사
shutil.copytree(source_dir, local_dir, dirs_exist_ok=True)

print("데이터를 로컬 환경에 복사하여 저장하였습니다.")


In [ ]:
import os
import shutil
from tqdm import tqdm

# Define the root directory of your data
data_root = "/kaggle/input/ucf-crime-dataset/Train"

# Get a list of all class directories
class_dirs = [os.path.join(data_root, class_name) for class_name in os.listdir(data_root)]

# Iterate through class directories
for class_dir in class_dirs:
    # Get a list of all image files in the class directory
    image_files = [f for f in os.listdir(class_dir) if f.endswith('.png')]

    # Create a subdirectory for each video
    for image_file in tqdm(image_files, desc=f"Processing {os.path.basename(class_dir)}"):
        video_name, frame_info = image_file.rsplit("_x264_", 1)
        video_dir = os.path.join(class_dir, video_name+'_x264_'+frame_info)
        if not os.path.exists(video_dir):
            os.makedirs(video_dir)

        # Move the image file to the video subdirectory
        src_path = os.path.join(class_dir, image_file)
        dst_path = os.path.join(video_dir, image_file)
        shutil.move(src_path, dst_path)


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, slow_fast_ratio=0.8, transform=None):
        self.root_dir = root_dir
        self.slow_fast_ratio = slow_fast_ratio
        self.transform = transform
        self.video_paths = self.collect_video_paths()

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        frames = self.load_frames(video_path)

        if frames is None:  # Skip videos with no frames
            return None

        # Split frames into slow and fast
        total_frames = len(frames)
        slow_frame_end = int(total_frames * self.slow_fast_ratio)
        slow_frames = frames[:slow_frame_end]
        fast_frames = frames[slow_frame_end:]

        # Apply transform to each frame
        if self.transform:
            slow_frames = [self.transform(frame) for frame in slow_frames]
            fast_frames = [self.transform(frame) for frame in fast_frames]

        return slow_frames, fast_frames

    def collect_video_paths(self):
        video_paths = []
        classes = os.listdir(self.root_dir)
        for class_name in classes:
            class_dir = os.path.join(self.root_dir, class_name)
            for img_file in os.listdir(class_dir):
                video_path = os.path.join(class_dir, img_file)
                if os.path.isdir(video_path):  # 디렉토리일 경우에만 추가
                    video_paths.append(video_path)
        return video_paths

    def load_frames(self, video_path):
        frames = []
        frame_files = sorted(os.listdir(video_path))

        if not frame_files:  # Skip videos with no frames
            return None

        for frame_file in frame_files:
            frame_path = os.path.join(video_path, frame_file)
            frame = Image.open(frame_path)
            frames.append(frame)
        return frames





In [ ]:
# Define the root directory of your image data
data_root = "/kaggle/input/ucf-crime-dataset/Train"

# Create an instance of CustomDataset
dataset = CustomDataset(data_root, slow_fast_ratio=0.8, transform=frame_transform)

# Create a DataLoader
batch_size = 4
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Iterate through the dataloader and process each batch
for batch in dataloader:
    if batch is None:
        continue

    slow_frames, fast_frames = batch
    # Now you can use these slow_frames and fast_frames as inputs to your SlowFast model
    # and perform video classification or action recognition



In [ ]:
frame_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
total_data_count = len(dataset)
print(f"Total number of data: {total_data_count}")


import matplotlib.pyplot as plt
import random

# Choose a random index for the sample image
sample_idx = random.randint(0, len(dataset) - 1)

# Get the sample image from the dataset
sample_image = dataset[sample_idx]

# Convert the tensor image to numpy array and transpose dimensions
sample_image = sample_image.permute(1, 2, 0).numpy()

# Denormalize the image
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
sample_image = sample_image * std + mean
sample_image = sample_image.clip(0, 1)

# Display the sample image using matplotlib
plt.figure(figsize=(8, 8))
plt.imshow(sample_image)
plt.axis('off')
plt.show()


In [ ]:
batch_size = 4

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

for batch_idx, batch in enumerate(dataloader):
    if batch_idx >= 10:
        break
    print(f"Batch {batch_idx+1} shape:", batch.shape)


In [ ]:

# Set the model to evaluation mode
model.eval()

# Iterate through the dataloader and process each batch
for batch in dataloader:
    slow_frames, fast_frames = batch

    # Process the slow_frames and fast_frames as needed
    # Here, you can directly use the model on frames without the feature_extractor
    with torch.no_grad():
        slow_outputs, fast_outputs = model(slow_frames), model(fast_frames)

    # Now you can use these outputs for further processing or analysis
    # For example, you can compute predictions or perform action recognition

In [ ]:
# Assuming you have already created your train_dataset and train_dataloader

# Get a batch from the train_dataloader
batch = next(iter(train_dataloader))

# Extract the inputs and labels from the batch
inputs, labels = batch

# Print information about the batch
print("Batch Inputs Shape:", inputs[0].shape)  # Shape of slow frames
print("Batch Inputs Shape:", inputs[1].shape)  # Shape of fast frames
print("Batch Labels:", labels)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from transformers import SlowFastFeatureExtractor, SlowFastForClassification

# Define your CustomDataset class (assuming you've already defined it)
class CustomDataset(Dataset):
    # ... your CustomDataset implementation ...

# Initialize the SlowFast feature extractor
feature_extractor = SlowFastFeatureExtractor.from_pretrained("facebookresearch/slowfast")

# Create an instance of the SlowFastForClassification model
num_classes = len(classes)  # Number of classes in your dataset
model = SlowFastForClassification.from_pretrained("facebookresearch/slowfast-8x8-r50", num_labels=num_classes)

# Define the transformation for individual frames
frame_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create an instance of your CustomDataset
dataset = CustomDataset(data_path, classes, max_video_num, transform=frame_transform)

# Create a DataLoader
batch_size = 4  # Adjust as needed
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10  # Adjust as needed

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in dataloader:
        slow_features = feature_extractor(images=inputs[0])
        fast_features = feature_extractor(images=inputs[1])

        optimizer.zero_grad()

        # Forward pass
        outputs = model(slow_features['slow'], fast_features['fast']).logits
        loss = criterion(outputs, labels)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print training loss for the epoch
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {running_loss / len(dataloader)}")

print("Training finished")


In [ ]:
import pickle

# Save DataLoader
with open('dataloader_slowfast.pkl', 'wb') as f:
    pickle.dump(dataloader, f)

In [ ]:
import os

data_path = '/kaggle/input/ucf-crime-dataset'  # 데이터 경로
file_path = os.path.join(data_path, 'Train/Abuse/Abuse003_x264_1.png')  # 파일 경로 생성

if os.path.exists(file_path):
    print(f"File '{file_path}' exists.")
else:
    print(f"File '{file_path}' does not exist.")


In [ ]:
import os

data_path = '/kaggle/input/ucf-crime-dataset'  # 데이터 경로
file_path = os.path.join(data_path, 'Train/Abuse/Abuse001_x264_10.png')  # 파일 경로 생성

if os.path.exists(file_path):
    print(f"File '{file_path}' exists.")
else:
    print(f"File '{file_path}' does not exist.")

In [ ]:
/kaggle/input/ucf-crime-dataset/Train/Abuse/Abuse001_x264_10.png

In [ ]:
pip install fvcore


In [ ]:
pip install slowfast


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from fvcore.common.checkpoint import Checkpointer
from slowfast.models import build_model


### SLOWFAST_8x8_R50

In [ ]:
import torch
# Choose the `slowfast_r50` model
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)

In [ ]:
import urllib.request

url = "https://dl.fbaipublicfiles.com/pyslowfast/model_zoo/kinetics400/SLOWFAST_8x8_R50.pkl"
filename = "SLOWFAST_8x8_R50.pkl"
urllib.request.urlretrieve(url, filename)
print(f"Downloaded {filename}")

url = "https://raw.githubusercontent.com/facebookresearch/SlowFast/main/configs/Kinetics/SLOWFAST_8x8_R50.yaml"
file_name = "SLOWFAST_8x8_R50.yaml"

urllib.request.urlretrieve(url, file_name)
print(f"{file_name} downloaded!")

In [ ]:
import torch
# Choose the `slowfast_r50` model
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)

In [ ]:
pip install av

In [ ]:
from typing import Dict
import json
import urllib
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)

In [ ]:
# Set to GPU or CPU
device = "cuda"
model = model.eval()
model = model.to(device)

In [ ]:
json_url = "https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json"
json_filename = "kinetics_classnames.json"
try: urllib.URLopener().retrieve(json_url, json_filename)
except: urllib.request.urlretrieve(json_url, json_filename)

In [ ]:
with open(json_filename, "r") as f:
    kinetics_classnames = json.load(f)

# Create an id to label name mapping
kinetics_id_to_classname = {}
for k, v in kinetics_classnames.items():
    kinetics_id_to_classname[v] = str(k).replace('"', "")

In [ ]:
kinetics_id_to_classname

In [ ]:
from torchvision.transforms import Compose, ToTensor, Normalize
from torch.utils.data import DataLoader

# 기존 DataLoader에서 이미지 파일만 추출하여 리스트에 담습니다.
images = []
for batch in dataloader:
    _, _, _, imgs = batch
    images.extend(imgs)

# 새로운 변환을 정의합니다.
new_transform = Compose([
    ToTensor(),  # 이미지를 텐서로 변환
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 이미지 정규화
])

# 이미지에 변환을 적용합니다.
transformed_images = [new_transform(img) for img in images]

# 변환된 이미지를 새로운 DataLoader에 담습니다.
new_dataloader = DataLoader(transformed_images, batch_size=batch_size, shuffle=True)


In [ ]:
from PIL import Image, UnidentifiedImageError
import os

def process_image(img_path):
    try:
        # 파일 이름에서 공통 부분과 프레임 순서 추출
        video_name, frame_info = os.path.basename(img_path).split("_x264_")
        frame_num, _ = os.path.splitext(frame_info)
        frame_num = int(frame_num)

        # 이미지 데이터 읽어오기
        img = Image.open(img_path)

        return video_name, frame_num, img
    except UnidentifiedImageError as e:
        print(f"Error processing {img_path}: {e}")
        return None, None, None




In [ ]:
classes = ['Explosion', 'Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Fighting', 'NormalVideos']
data_path = '/content/drive/MyDrive/Colab Notebooks/Final_team_3/Data/archive (2)/Train'

input_data = []

# 각 라벨에 대해 처리
for label in classes:
    label_dir = os.path.join(data_path, label)

    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)

        video_name, frame_num, img = process_image(img_path)
        if video_name is not None:
            input_data.append((label, video_name, frame_num, img))
            if frame_num <300:
                print(label, video_name, frame_num, img)

In [ ]:
from PIL import Image, UnidentifiedImageError
import os

classes = ['Explosion', 'Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Fighting','NormalVideos']

data_path = '/content/drive/MyDrive/Colab Notebooks/Final_team_3/Data/archive (2)/Train'

input_data = []

# 각 라벨에 대해 처리
for label in classes:
    label_dir = os.path.join(data_path, label)

    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)

        try:
            # 파일 이름에서 공통 부분과 프레임 순서 추출
            video_name, frame_info = img_file.split("_x264_")
            frame_num, _ = os.path.splitext(frame_info)
            frame_num = int(frame_num)

            # 이미지 데이터 읽어오기
            img = Image.open(img_path)

            input_data.append((label, video_name, frame_num, img))
            if frame_num > 10000:
              print(label, video_name, frame_num, img)

        except UnidentifiedImageError as e:
            print(f"Error processing {img_path}: {e}")




Error processing /content/drive/MyDrive/Colab Notebooks/Final_team_3/Data/archive (2)/Train/Arrest/Arrest050_x264_3100.png

Error processing /content/drive/MyDrive/Colab Notebooks/Final_team_3/Data/archive (2)/Train/Arrest/Arrest050_x264_310.png

Error processing /content/drive/MyDrive/Colab Notebooks/Final_team_3/Data/archive (2)/Train/Arrest/Arrest050_x264_3090.png

Error processing /content/drive/MyDrive/Colab Notebooks/Final_team_3/Data/archive (2)/Train/Arrest/Arrest050_x264_3080.png

Error processing /content/drive/MyDrive/Colab Notebooks/Final_team_3/Data/archive (2)/Train/Arrest/Arrest050_x264_3110.png

In [ ]:
import pickle

# input_data 저장
with open('input_data.pkl', 'wb') as f:
    pickle.dump(input_data, f)

# 저장한 input_data 로드
with open('input_data.pkl', 'rb') as f:
    loaded_input_data = pickle.load(f)

In [ ]:
# 저장한 input_data 로드
with open('input_data.pkl', 'rb') as f:
    loaded_input_data = pickle.load(f)

In [ ]:
from google.colab import files

# Colab 파일 다운로드
files.download('input_data.pkl')

In [ ]:
from PIL import Image
import os
import torchvision.transforms as transforms
from torchvision.io import read_image
from torchvision.transforms import functional as F

data_path = '/content/drive/MyDrive/Colab Notebooks/Final_team_3/Data/archive (2)/Train'

input_data = []

# 각 라벨에 대해 처리
for label in os.listdir(data_path):
    label_dir = os.path.join(data_path, label)

    for img_file in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_file)

        # 파일 이름에서 공통 부분과 프레임 순서 추출
        video_name, frame_info = img_file.split("_x264_")
        frame_num, _ = os.path.splitext(frame_info)
        frame_num = int(frame_num)

        # 이미지 데이터 읽어오기
        img = Image.open(img_path)

        input_data.append((label, video_name, frame_num, img))

# SlowFast 모델에 입력으로 활용하기 위한 전처리
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 이미지 크기 조정
    transforms.ToTensor(),  # 텐서로 변환
])

# SlowFast 모델의 입력 형식에 맞게 데이터를 처리
slow_pathway_data = {}
fast_pathway_data = {}
for label, video_name, frame_num, img in input_data:
    img_tensor = transform(img)
    if label not in slow_pathway_data:
        slow_pathway_data[label] = []
    if label not in fast_pathway_data:
        fast_pathway_data[label] = []

    if frame_num % 16 == 0:  # Slow pathway
        slow_pathway_data[label].append(img_tensor)
    else:  # Fast pathway
        fast_pathway_data[label].append(img_tensor)

# SlowFast 모델에 입력으로 활용할 데이터 생성
slowfast_input_data = []
for label in slow_pathway_data:
    slow_tensors = slow_pathway_data[label]
    fast_tensors = fast_pathway_data[label]
    for i in range(len(slow_tensors)):
        slow_tensor = torch.stack(slow_tensors[i:i+8])  # 8 프레임씩 묶음
        fast_tensor = torch.stack(fast_tensors[i:i+8])
        slowfast_input_data.append((slow_tensor, fast_tensor, label))





In [ ]:
classes = ['Explosion', 'Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Fighting','NormalVideos']

In [ ]:
for label in classes:
    label_dir = os.path.join(data_path, label)
    print(label_dir)


In [ ]:
label_dir